In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Create Fully Connected Network

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1= nn.Linear(input_size, 50)
        self.fc2= nn.Linear(50, num_classes)
        
    def forward(self, x):
        x= F.relu(self.fc1(x))
        x= self.fc2(x)
        return x

# Set Devices

In [4]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [5]:
input_size= 784
num_classes= 10
learning_rate= 0.001
batch_size= 64
num_epochs= 1

# Load Data

In [6]:
train_dataset= datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download= True )

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 6180004.93it/s] 


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1066532.49it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 8245660.88it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3480184.28it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [8]:
train_loader= DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle= True)
test_dataset= datasets.MNIST(root='dataset/', train=False, transform= transforms.ToTensor(), download=True)
test_loader= DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle= True)

# Initialize Network

In [9]:
model= NN(input_size=input_size, num_classes= num_classes).to(device)

# Loss and Optimizer

In [10]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(), lr= learning_rate)

# Train Network

In [14]:
for epoch in range(num_epochs):
    for batch_idx,(data, targets) in enumerate(train_loader):
        data= data.to(device=device)
        targets= targets.to(device=device)
        
        data=data.reshape(data.shape[0], -1)
        
        #forward
        scores= model(data)
        loss= criterion(scores, targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent or adam step
        optimizer.step()
        
        

# Check accuracy on training & Test to see how good our model

In [17]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking the accuracy on training data")
    else:
        print("Checking the accuracy on Test data")
        
    num_correct= 0
    num_samples= 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x= x.to(device=device)
            y= y.to(device=device)
            x= x.reshape(x.shape[0], -1)
            
            scores= model(x)
            _, predictions= scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:2f}')
    
    model.train()
            
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking the accuracy on training data
Got 55713/60000 with accuracy 92.855000
Checking the accuracy on Test data
Got 9271/10000 with accuracy 92.710000
